In [ ]:
import pandas as pd
from labels import LABELS

In [ ]:
data = "/teamspace/studios/this_studio/max/dataset_resume.csv"
data = "./dataset_resume.csv"
df = pd.read_csv(data, usecols=LABELS + ['description', 'resume'])


In [ ]:
import numpy as np


def setup(df_orig: pd.DataFrame) -> pd.DataFrame:
    df = df_orig.copy()
    df = df.fillna(0)

    for index, row in df.iterrows():
        # s'il y a un résumé, on l'utilise comme description
        df.loc[index, "description"] = (
                row.resume
                if isinstance(row.resume, str)
                else row.description
            )

    df['label'] = df[LABELS].values.tolist()
    df = df.rename(columns={'description': 'text'})
    label = [list_label for list_label in df.label]
    return df.text.tolist(), label#.astype('float32')

features = df[LABELS]

texts, labels = setup(df)

In [ ]:
import torch
from transformers import CamembertTokenizer, AutoTokenizer
from transformers import CamembertForSequenceClassification, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=.2, random_state=42)

In [ ]:
checkpoint = "almanach/camembert-base"
tokenizer = CamembertTokenizer.from_pretrained(checkpoint)
model = CamembertForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels[0]), problem_type="multi_label_classification")

In [ ]:
class EventDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = torch.tensor(self.labels[index])

        encoding = self.tokenizer(text, 
                                  padding="max_length", 
                                  max_length=self.max_len,
                                  truncation=True,
                                  return_tensors="pt")

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label
        }

In [ ]:
train_ds = EventDataset(train_texts, train_labels, tokenizer)
val_ds = EventDataset(val_texts, val_labels, tokenizer)

In [ ]:
# Multi-Label Classification Evaluation Metrics
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss
from transformers import EvalPrediction
import torch


def multi_labels_metrics(predictions, labels, threshold=0.3):
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions))

  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs>=threshold)] = 1
  y_true = labels

  f1 = f1_score(y_true, y_pred, average = 'macro')
  roc_auc = roc_auc_score(y_true, y_pred, average = 'macro')
  hamming = hamming_loss(y_true, y_pred)

  metrics = {
      "roc_auc": roc_auc,
      "hamming_loss": hamming,
      "f1": f1
  }

  return metrics

def compute_metrics(p:EvalPrediction):
  preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions

  result = multi_labels_metrics(predictions=preds,
                                labels=p.label_ids)

  return result

In [ ]:
# Training Arguments
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

args = TrainingArguments(
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    output_dir = './results',
    num_train_epochs=300,
    save_steps=1000,
    save_total_limit=2,
    eval_steps=10,
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    load_best_model_at_end=True
)

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=train_ds,
                  eval_dataset = val_ds, 
                  compute_metrics=compute_metrics,
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
                  )

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model("/teamspace/studios/this_studio/max/camembert-tourism-events")

In [ ]:
import pickle
with open("/teamspace/studios/this_studio/max/features.pkl", "wb") as f:
  pickle.dump(features, f)

In [ ]:
text = """Après de multiples tournées à succès et plus de 2 millions de billets vendus, Messmer, connu et reconnu comme le Maître Mondial de l’Hypnose revient près de chez vous !

Dans 13Hz, Messmer vous invite à entrer dans son mystérieux et hilarant univers où la frontière entre la réalité et l’illusion s’efface, pour diriger vos pensées vers des territoires inconnus.

Avec sa présence charismatique inégalée et son talent exceptionnel, le recordman en hypnose collective avec 1066 personnes hypnotisées en moins de 5 minutes, vous plonge au cœur de vos pensées les plus profondes avant de vous guider à travers un jeu subtil d’ondes cérébrales à 13Hz.

Le fascinateur vous entraîne vers un état de conscience unique où la volonté et le contrôle de nos vies prennent une nouvelle dimension.

Osez découvrir l’expérience Messmer, où la maîtrise de soi et la fascination se rencontrent."""

encoding = tokenizer(text, return_tensors='pt')
encoding.to(trainer.model.device)

outputs = trainer.model(**encoding)

In [ ]:
import json
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0].cpu()).detach().numpy()
preds = np.zeros(probs.shape)
##preds[np.where(probs>=0.3)] = 1


def sort_dict_by_value(dict1):
  """Trie un dictionnaire par ordre décroissant des valeurs."""
  sorted_dict = {x: y for x, y in sorted(dict1.items(), key=lambda item: item[1], reverse=True)}
  return sorted_dict
  
preds = '{'
for idx, label in enumerate(LABELS):
    preds += f'"{label}": {probs[idx]},'

preds = preds.strip(',')
preds += '}'
preds = json.loads(preds)
sort_dict_by_value(preds)

# Utilisation d'un modèle enregistré en local

In [47]:
model = CamembertForSequenceClassification.from_pretrained('./camembert-tourism-events')

In [48]:
text = """Après de multiples tournées à succès et plus de 2 millions de billets vendus, Messmer, connu et reconnu comme le Maître Mondial de l’Hypnose revient près de chez vous !

Dans 13Hz, Messmer vous invite à entrer dans son mystérieux et hilarant univers où la frontière entre la réalité et l’illusion s’efface, pour diriger vos pensées vers des territoires inconnus.

Avec sa présence charismatique inégalée et son talent exceptionnel, le recordman en hypnose collective avec 1066 personnes hypnotisées en moins de 5 minutes, vous plonge au cœur de vos pensées les plus profondes avant de vous guider à travers un jeu subtil d’ondes cérébrales à 13Hz.

Le fascinateur vous entraîne vers un état de conscience unique où la volonté et le contrôle de nos vies prennent une nouvelle dimension.

Osez découvrir l’expérience Messmer, où la maîtrise de soi et la fascination se rencontrent."""

encoding = tokenizer(text, return_tensors='pt')
encoding.to(trainer.model.device)

outputs = trainer.model(**encoding)

In [49]:
import json
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0].cpu()).detach().numpy()
preds = np.zeros(probs.shape)
##preds[np.where(probs>=0.3)] = 1


def sort_dict_by_value(dict1):
  """Trie un dictionnaire par ordre décroissant des valeurs."""
  sorted_dict = {x: y for x, y in sorted(dict1.items(), key=lambda item: item[1], reverse=True)}
  return sorted_dict
  
preds = '{'
for idx, label in enumerate(LABELS):
    preds += f'"{label}": {probs[idx]},'

preds = preds.strip(',')
preds += '}'
preds = json.loads(preds)
sort_dict_by_value(preds)

{'Brocante': 0.5014354586601257,
 'Festival': 0.496290385723114,
 'Culture': 0.49298417568206787,
 'Exposition': 0.49059388041496277,
 'Action': 0.4885813593864441,
 'Balade': 0.4874850809574127,
 'Gastronomie': 0.48692554235458374,
 'Fête': 0.4864422380924225,
 'Famille': 0.4856192469596863,
 'Spectacle': 0.48173367977142334,
 'Jeu': 0.4791945219039917,
 'Santé': 0.47722187638282776,
 'Théatre': 0.47510436177253723,
 'Marché': 0.47184333205223083,
 'Concert': 0.47170743346214294,
 'Atelier': 0.46970421075820923,
 'Détente': 0.46968236565589905,
 'Danse': 0.4673236906528473,
 'Visite': 0.4669744670391083,
 'Conférence': 0.4638778865337372,
 'Environnement': 0.46037420630455017,
 'Art': 0.45369207859039307,
 'Histoire': 0.45325541496276855,
 'Sport': 0.45077407360076904}